Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
      (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('\nMinibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
              valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 3.968358
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%

Minibatch loss at step 50: 2.291015
Minibatch accuracy: 18.8%
Validation accuracy: 23.5%

Minibatch loss at step 100: 1.484002
Minibatch accuracy: 56.2%
Validation accuracy: 57.6%

Minibatch loss at step 150: 1.223457
Minibatch accuracy: 56.2%
Validation accuracy: 71.5%

Minibatch loss at step 200: 1.063030
Minibatch accuracy: 81.2%
Validation accuracy: 76.5%

Minibatch loss at step 250: 0.716814
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%

Minibatch loss at step 300: 0.876635
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%

Minibatch loss at step 350: 0.829865
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%

Minibatch loss at step 400: 0.720765
Minibatch accuracy: 81.2%
Validation accuracy: 74.9%

Minibatch loss at step 450: 0.509839
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%

Minibatch loss at step 500: 1.032833
Minibatch accuracy: 62.5%
Validation accura

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.max_pool(
            tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
            , [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.max_pool(
            tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
            , [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('\nMinibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
              valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 3.031647
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%

Minibatch loss at step 50: 1.972845
Minibatch accuracy: 31.2%
Validation accuracy: 29.9%

Minibatch loss at step 100: 1.532422
Minibatch accuracy: 50.0%
Validation accuracy: 60.9%

Minibatch loss at step 150: 1.116102
Minibatch accuracy: 62.5%
Validation accuracy: 75.3%

Minibatch loss at step 200: 1.226999
Minibatch accuracy: 62.5%
Validation accuracy: 78.0%

Minibatch loss at step 250: 0.705871
Minibatch accuracy: 68.8%
Validation accuracy: 77.4%

Minibatch loss at step 300: 0.967133
Minibatch accuracy: 75.0%
Validation accuracy: 81.0%

Minibatch loss at step 350: 0.832878
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%

Minibatch loss at step 400: 0.922405
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%

Minibatch loss at step 450: 0.333917
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%

Minibatch loss at step 500: 1.130067
Minibatch accuracy: 62.5%
Validation accur

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

## LeNet-5 architecture

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [5, 5, num_channels, 6], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([6]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [6, 6, 6, 16], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[16]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([6*6*16, 120], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[120]))
    
    layer3b_weights = tf.Variable(tf.truncated_normal([120, 84], stddev=0.1))
    layer3b_biases = tf.Variable(tf.constant(1.0, shape=[84]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [84, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    dropout_keep_prob = 1.0
  
    # Model.
    def model(data):
        conv = tf.nn.dropout(
            tf.nn.max_pool(
            tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
            , [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
            , dropout_keep_prob)
        hidden = tf.nn.relu(conv + layer1_biases)
        
        conv2 = tf.nn.dropout(
            tf.nn.max_pool(
            tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
            , [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
            , dropout_keep_prob)
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        shape = hidden2.get_shape().as_list()
        reshape = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden3 = tf.nn.dropout(
            tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            , dropout_keep_prob)
        hidden3b = tf.nn.dropout(
            tf.nn.relu(tf.matmul(hidden3, layer3b_weights) + layer3b_biases)
            , dropout_keep_prob)
        return tf.matmul(hidden3b, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 10001, 0.5)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('\nMinibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
              valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 4.602861
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%

Minibatch loss at step 50: 2.199670
Minibatch accuracy: 6.2%
Validation accuracy: 29.1%

Minibatch loss at step 100: 1.772775
Minibatch accuracy: 56.2%
Validation accuracy: 40.4%

Minibatch loss at step 150: 1.555467
Minibatch accuracy: 43.8%
Validation accuracy: 67.8%

Minibatch loss at step 200: 1.205133
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%

Minibatch loss at step 250: 0.655205
Minibatch accuracy: 75.0%
Validation accuracy: 75.1%

Minibatch loss at step 300: 0.939636
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%

Minibatch loss at step 350: 0.849920
Minibatch accuracy: 68.8%
Validation accuracy: 75.6%

Minibatch loss at step 400: 0.719055
Minibatch accuracy: 81.2%
Validation accuracy: 76.5%

Minibatch loss at step 450: 0.249256
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%

Minibatch loss at step 500: 1.025184
Minibatch accuracy: 62.5%
Validation accurac